In [1]:
from splunk_tools import SplunkTools
import asyncio
savedsearches = ["Windows Event For Service Disabled",
                 "Detect New Local Admin account",
                 "ESCU Network Share Discovery Via Dir Command Rule",
                 "Known Services Killed by Ransomware",
                 "Non Chrome Process Accessing Chrome Default Dir",
                 "Kerberoasting spn request with RC4 encryption",
                 "Clop Ransomware Known Service Name"]

splunk = SplunkTools(savedsearches, 1)

logtypes_list = [('wineventlog:system', '1'), ('wineventlog:system', '16'), ('wineventlog:system', '1085'), ('wineventlog:security', '5061'), ('wineventlog:system', '7000'), ('wineventlog:system', '566'), ('wineventlog:system', '107'), ('wineventlog:system', '55'), ('wineventlog:security', '4663'), ('wineventlog:system', '7038'), ('wineventlog:system', '24'), ('wineventlog:security', '4625'), ('wineventlog:system', '7031'), ('wineventlog:security', '4662'), ('wineventlog:system', '7040'), ('wineventlog:security', '4672'), ('wineventlog:system', '1112'), ('wineventlog:system', '8015'), ('wineventlog:system', '101'), ('wineventlog:security', '4702'), ('wineventlog:system', '44'), ('wineventlog:security', '4735'), ('wineventlog:system', '37'), ('wineventlog:security', '4697'), ('wineventlog:security', '5059'), ('wineventlog:system', '1500'), ('wineventlog:system', '7036'), ('wineventlog:system', '1501'), ('wineventlog:system', '131'), ('wineventlog:system', '187'), ('wineventlog:security', '4732'), ('wineventlog:system', '103'), ('wineventlog:system', '32'), ('wineventlog:system', '35'), ('wineventlog:security', '4799'), ('wineventlog:security', '5058'), ('wineventlog:security', '4648'), ('wineventlog:security', '5379'), ('wineventlog:security', '4634'), ('wineventlog:system', '7'), ('wineventlog:system', '129'), ('wineventlog:security', '5038'), ('wineventlog:security', '4907'), ('wineventlog:security', '4624'), ('wineventlog:system', '27'), ('wineventlog:system', '10016'), ('wineventlog:system', '7045'), ('wineventlog:system', '6'), ('wineventlog:system', '108'), ('wineventlog:security', '4698'), ('wineventlog:security', '5140'), ('wineventlog:system', '12'), ('wineventlog:security', '4627'), ('wineventlog:system', '6013'), ('wineventlog:security', '4769')]
# find one sample of each logtype in splunk in all time and save as raw
async def find(logtype, eventcode):
    query = f"index=main source={logtype} EventCode={eventcode} | head 1"
    return await splunk.execute_query(f"find_one_{logtype}_{eventcode}", query, 0, "now")
    
results = await asyncio.gather(*[find(logtype, eventcode) for logtype, eventcode in logtypes_list])

/home/shouei/anaconda3/envs/py38/lib/python3.8/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '132.72.81.150'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [2]:
log_samples = {}
for metric, result in results:
    result = result[0]
    source = result['source'] 
    eventcode = result['EventCode']
    raw = result['_raw']
    log_samples[(source, eventcode)] = raw

log_samples   

{('WinEventLog:System',
  '1'): '03/26/2025 08:00:17 PM\nLogName=System\nEventCode=1\nEventType=4\nComputerName=LB-101-18.auth.ad.bgu.ac.il\nUser=NOT_TRANSLATED\nSid=S-1-5-19\nSidType=0\nSourceName=Microsoft-Windows-Power-Troubleshooter\nType=Information\nRecordNumber=42292\nKeywords=None\nTaskCategory=None\nOpCode=Info\nMessage=The system has returned from a low power state.\r\n\r\nSleep Time: \u200e2025\u200e-\u200e03\u200e-\u200e26T18:00:06.032475200Z\r\nWake Time: \u200e2025\u200e-\u200e03\u200e-\u200e26T18:00:15.926370100Z\r\n\r\nWake Source: Device -Intel(R) Ethernet Connection (17) I219-LM',
 ('WinEventLog:System',
  '16'): '03/26/2025 09:59:57 PM\nLogName=System\nEventCode=16\nEventType=4\nComputerName=CYB301-4.auth.ad.bgu.ac.il\nUser=NOT_TRANSLATED\nSid=S-1-5-18\nSidType=0\nSourceName=Microsoft-Windows-Kernel-General\nType=Information\nRecordNumber=137647\nKeywords=None\nTaskCategory=None\nOpCode=Info\nMessage=The access history in hive \\??\\C:\\ProgramData\\Microsoft\\Window

In [3]:
from datetime import datetime
import re
# validate raw logs by send them to splunk and search for them
async def validate(source, eventcode, raw):
    # change date using regex
    now = datetime.now()
    raw = re.sub(r'^\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2} [APM]{2}', now.strftime("%m/%d/%Y %I:%M:%S %p"), raw)
    # send raw to splunk
    splunk.write_logs_to_monitor([raw], source.lower())
    # search for the raw in splunk
    query = f"index=main source={source} EventCode={eventcode} earliest=-5m host=dt-splunk"
    return await splunk.execute_query(f"validate_{source}_{eventcode}", query, 0, "now")

results = await asyncio.gather(*[validate(source, eventcode, raw) for (source, eventcode), raw in log_samples.items()])


In [4]:
results

[(QueryMetrics(search_name='validate_WinEventLog:System_1', results_count=1, execution_time=2.391, cpu=0, io_metrics={'read_count': 0, 'write_count': 0, 'read_bytes': 0, 'write_bytes': 0}, start_time=0.0, end_time=0.0),
  [{'EventCode': '1',
    'Message': 'The system has returned from a low power state.\r\n\r\nSleep Time: \u200e2025\u200e-\u200e03\u200e-\u200e26T18:00:06.032475200Z\r\nWake Time: \u200e2025\u200e-\u200e03\u200e-\u200e26T18:00:15.926370100Z\r\n\r\nWake Source: Device -Intel(R) Ethernet Connection (17) I219-LM',
    '_bkt': 'main~1877~5E9D06A2-F80B-43DF-85C0-53D15F89256C',
    '_cd': '1877:24333881',
    '_indextime': '1743019723',
    '_pre_msg': '03/26/2025 10:08:42 PM\nLogName=System\nEventCode=1\nEventType=4\nComputerName=LB-101-18.auth.ad.bgu.ac.il\nUser=NOT_TRANSLATED\nSid=S-1-5-19\nSidType=0\nSourceName=Microsoft-Windows-Power-Troubleshooter\nType=Information\nRecordNumber=42292\nKeywords=None\nTaskCategory=None\nOpCode=Info',
    '_raw': '03/26/2025 10:08:42 PM\n